# wRMSD calculation

This notebook analyzes the results of molecular dynamics simulations of antibody molecules.
It calculates the root-mean-square deviation (RMSD) between the candidate antibody and the reference antibody,
and uses this to calculate the weighted RMSD (wRMSD) based on the accessibility and movability of each residue.

The script requires the MDAnalysis library to be installed.

In [1]:
import os
from functools import reduce

import numpy as np

import MDAnalysis as mda

In [2]:
# Define paths to the reference and candidate antibody simulation data
PATH_REFERENCE_AB = 'md_simulations/remicade_humanization/remicade/'
PATH_CANDIDATE_AB = 'md_simulations/remicade_humanization/candidate_Aa/'

# Define the CDR residues for the antibodies
CDRS = {
    'H1': (26, 33),    # 26-32
    'H2': (52, 60),    # 52-59
    'H3': (101, 110),  # 101-109
    'L1': (144, 155),  # 24-34
    'L2': (170, 177),  # 50-56
    'L3': (209, 218),  # 89-97
} 

In [3]:
# Load the reference and candidate antibody simulation data
reference_ab = mda.Universe(os.path.join(PATH_REFERENCE_AB, 'structures/newbox.gro'),
                            os.path.join(PATH_REFERENCE_AB, 'prod_test/prod_test_noPBC.xtc')
                           )
candidate_ab = mda.Universe(os.path.join(PATH_CANDIDATE_AB, 'structures/newbox.gro'),
                            os.path.join(PATH_CANDIDATE_AB, 'prod_test/prod_test_noPBC.xtc')
                           )

reference_ab, candidate_ab

(<Universe with 3445 atoms>, <Universe with 3476 atoms>)

In [4]:
# Select the CDR residues and their alpha carbons for the reference and candidate antibodies
reference_cdr = sum([reference_ab.residues[val[0] - 1:val[1] - 1].atoms for val in CDRS.values()])
reference_cdr_ca = reference_cdr.select_atoms('name CA')

candidate_cdr = sum([candidate_ab.residues[val[0] - 1:val[1] - 1].atoms for val in CDRS.values()])
candidate_cdr_ca = candidate_cdr.select_atoms('name CA')

reference_cdr_ca, candidate_cdr_ca

(<AtomGroup with 51 atoms>, <AtomGroup with 51 atoms>)

In [5]:
# Define a function to read the solvent-accessible surface area (SASA) data from a gromacs-generated file
def read_sasa_xvg(filepath):
    sasa = []
    with open(filepath, 'r') as file:
        for line in file:
            if line.startswith('#') or line.startswith('@'):
                continue

            row = line.strip().split()
            sasa.append([float(val) for val in row[2:]])

    return np.array(sasa)

# Load the SASA data for the reference and candidate antibodies
reference_sasa = read_sasa_xvg(os.path.join(PATH_REFERENCE_AB, 'sasa/sasa.xvg'))
candidate_sasa = read_sasa_xvg(os.path.join(PATH_CANDIDATE_AB, 'sasa/sasa.xvg'))

reference_sasa.shape

(5001, 51)

$$RMSD_i = \sqrt{\frac{\sum_j \left(d_{ij} - d_{ij}^0 \right)^2}{n}}$$

$$wRMSD_i = \sum_i \frac{w_{i-accessibility} w_{i-movability} RMSD_i}{N}$$

$$w_{i-accessibility} = 1 / \left(\frac{S_i}{S_{i.avg}} \right)$$

$$w_{i-movability} = 1 / \left(\frac{RMSD_i^0}{RMSD_{i.avg}^0} \right)$$

In [6]:
# Define a function to calculate the distance matrix from a set of coordinates
def coordinates2distance_matrix(coordinates):
    return np.sqrt(np.sum((coordinates[:, np.newaxis, :] - coordinates[np.newaxis, :, :]) ** 2, axis=-1))


n = len(reference_cdr_ca)
rmsd = []

# Calculate the RMSD for each frame of the trajectories
for ts_ref, ts_cand in zip(reference_ab.trajectory, 
                           candidate_ab.trajectory,
                          ):
    distance_matrix_ref = coordinates2distance_matrix(ts_ref[reference_cdr_ca.ix])
    distance_matrix_cand = coordinates2distance_matrix(ts_cand[candidate_cdr_ca.ix])
    rmsd.append(np.sqrt(np.sum((distance_matrix_cand - distance_matrix_ref) ** 2, axis=1) / (n - 1)))

rmsd = np.array(rmsd)
print(rmsd.shape)
rmsd

(5001, 51)


array([[1.973204  , 2.1445262 , 2.0732281 , ..., 1.242502  , 1.0991721 ,
        0.9360441 ],
       [1.7039387 , 1.7378993 , 1.8898654 , ..., 1.0854164 , 1.0500093 ,
        0.75137657],
       [1.591384  , 1.6245047 , 1.6598525 , ..., 1.1067663 , 0.7925387 ,
        0.67691195],
       ...,
       [2.0031538 , 2.1413727 , 1.6950476 , ..., 1.1522515 , 1.0573719 ,
        0.78408647],
       [2.1550474 , 1.9671072 , 2.0468764 , ..., 1.2702253 , 1.2794461 ,
        0.90414405],
       [1.9909649 , 1.7221991 , 1.995687  , ..., 1.4017806 , 1.2498034 ,
        0.9345446 ]], dtype=float32)